# Inleiding MongoDB

* **Voorkennis:** [Inleiding Jupyter](Inleiding-Jupyter.ipynb)

In dit notebook geven we een aantal voorbeelden van MongoDB, aan de hand van de MongoDB shell (de commando-regel).
Hiermee kun je oefenen met zoekopdrachten e.d.
Het gebruik van een MongoDB-database in een Python-toepassing hebben we uitgewerkt in de andere Notebooks.
Meer informatie over de MongoDB shell-opdrachten vind je via: [Mongo shell](https://docs.mongodb.com/manual/mongo/).

In dit notebook moet je per cell de mongo-shell opstarten; hiervoor gebruiken je de opdracht `mongo demo --quiet`.
Daarna kun je één of meer mongo-shell opdrachten geven.

De onderstaande opdracht is nodig als initialisatie in sommige omgevingen.

In [1]:
import os
os.environ["PATH"] += ":/usr/local/bin"

## Importeren voorbeeld-database

Als eerste opdracht lezen we de voorbeeld-database in, met `mongoimport`. Dit hoeft je maar één keer per sessie te doen. Je kunt hiermee ook de database herstellen als je deze verziekt hebt.

In de onderstaande opdracht importeren we de *collection* `contacts` uit het bestand `adressen.json`.

In [2]:
%%bash
mongoimport -d demo --drop -c contacts adressen.json

2019-08-25T14:36:12.397+0200	connected to: localhost
2019-08-25T14:36:12.400+0200	dropping: demo.contacts
2019-08-25T14:36:12.599+0200	imported 7 documents


Met de onderstaande opdracht bekijk je het bestand `adressen.json`.

**Ga na**:

* welke documenten hebben geen email-adres?
* welke documenten hebben meerdere email-adressen?
* welke documenten hebben geen adres?
* welke documenten hebben geen postcode in het adres?


In [3]:
%%bash
cat adressen.json

{"name": "Hans de Boer",
 "email": "hdb@example.com",
 "tel": "06-1290 8746"
}
{"name": "Anna Verschuur",
 "email": "anna@hotmail.com",
 "address": {"street": "Noorderkade 102",
             "city": "Amsterdam",
             "postcode": "1000 AA"
            }
}
{"name": "F.G. Schuitema",
 "address": {"street": "Eikenlaan 23",
             "city": "Amsterdam",
             "postcode": "1001 AB"
            }
}
{"name": "Adrie Vierhuis",
 "email": "a34huis@gmail.com",
 "address": {"street": "Beukenweg 12",
             "city": "Rotterdam",
             "postcode": "2001 BC"
            },
  "tel": "010-123 123 9"
}
{"name": "Joop de Zwart",
 "email": "zwartejoop@ziggo.nl",
 "address": {"street": "Rozengracht 42",
             "city": "Rotterdam"}
}
{"name": "Gijs Bennekom",
 "email": "gijsbkom@ziggo.nl",
 "address": {"street": "Lijsterlaan 132",
             "city": "Rotterdam"}
}
{"name": "Leontien de Bruin ",
 "email":
 ["lhmdebruin@hotmail.com",
  "leontien134@tiscalimail.nl"
 ],
 "a

## Eerste zoekopdrachten

De zoekopdracht in de mongo-shell heeft de vorm: `db.collection.find(query-doc)`. Hierin is `query-doc` het zoek- document waarin een deel van de velden ingevuld is. De zoekopdracht vindt alle documenten die matchen met dit zoek-document.

Als je geen zoekdocument opgeeft, of een leeg zoekdocument (`{}`), dan krijg je alle documenten in de collection.

In [4]:
%%bash
mongo demo --quiet

db.contacts.find({})

{ "_id" : ObjectId("5d6280bc6f385c47986b6c52"), "name" : "Anna Verschuur", "email" : "anna@hotmail.com", "address" : { "street" : "Noorderkade 102", "city" : "Amsterdam", "postcode" : "1000 AA" } }
{ "_id" : ObjectId("5d6280bc6f385c47986b6c53"), "name" : "Joop de Zwart", "email" : "zwartejoop@ziggo.nl", "address" : { "street" : "Rozengracht 42", "city" : "Rotterdam" } }
{ "_id" : ObjectId("5d6280bc6f385c47986b6c54"), "name" : "Adrie Vierhuis", "email" : "a34huis@gmail.com", "address" : { "street" : "Beukenweg 12", "city" : "Rotterdam", "postcode" : "2001 BC" }, "tel" : "010-123 123 9" }
{ "_id" : ObjectId("5d6280bc6f385c47986b6c55"), "name" : "Gijs Bennekom", "email" : "gijsbkom@ziggo.nl", "address" : { "street" : "Lijsterlaan 132", "city" : "Rotterdam" } }
{ "_id" : ObjectId("5d6280bc6f385c47986b6c56"), "name" : "Leontien de Bruin ", "email" : [ "lhmdebruin@hotmail.com", "leontien134@tiscalimail.nl" ], "address" : { "street" : "Tulpstraat 17", "city" : "Amsterdam" } }
{ "_id" : Object

### Key: `_id`-veld

Merk op dat de geïmporteerde documenten een veld `_id` hebben: dit is de unieke key van het document die MongoDB automatisch aangemaakt heeft. Deze key is uniek voor alle databases en collections.

### Zoeken naar enkel document

Door een veld op te geven dat uniek is binnen de collectie, vind je een enkel document.

In [5]:
%%bash
mongo demo --quiet

db.contacts.find({"name": "Hans de Boer"})

{ "_id" : ObjectId("5d6280bc6f385c47986b6c57"), "name" : "Hans de Boer", "email" : "hdb@example.com", "tel" : "06-1290 8746" }


**Opdracht** zoek een document op basis van het email-veld.

In [6]:
%%bash
mongo demo --quiet

db.contacts.find({???})

2019-08-25T14:36:14.666+0200 E QUERY    [js] SyntaxError: invalid property id @(shell):1:18


**Opdracht** zoek een document op basis van het `_id`-veld.

In [7]:
%%bash
mongo demo --quiet

db.contacts.find({???})

2019-08-25T14:36:15.063+0200 E QUERY    [js] SyntaxError: invalid property id @(shell):1:18


## Projectie

Met "find" selecteren we een verzameling documenten. De algemene vorm is: `db.collection.find({<selection>}, {<projection>})`. Met het projectie-document geef je aan welke velden van deze documenten je in het resultaat wilt zien: een veld in het resultaat geeft je aan met `fieldname:1`; het weglaten van een veld geef je aan met `fieldname:0`. Afgezien van het `_id`-field, kun je in een projectie-document alleen "0"-en of "1"-en opgeven.

> In een SQL-querie geef je na de `SELECT` een opsomming van de kolommen in het resultaat: dit is de SQL-vorm van projectie.

**Voorbeeld**

In [8]:
%%bash
mongo demo --quiet

db.contacts.find({"name": "Hans de Boer"}, {"_id":0, "name":1, "email": 1})

{ "name" : "Hans de Boer", "email" : "hdb@example.com" }


**Opdracht.** De volgorde van de velden in het resultaat hangt af van de oorspronkelijke definitie van het document, niet van de volgorde van de velden in de projectie. **Ga dit na**, door onderstaande query aan te passen.

> Dit is een verschil met SQL: daar bepaalt de query de volgorde van de kolommen in het resultaat.

In [9]:
%%bash
mongo demo --quiet

db.contacts.find({"name": "Hans de Boer"}, {"_id":0, "name":1, "email": 1})

{ "name" : "Hans de Boer", "email" : "hdb@example.com" }


## Selectie in een samengestelde waarde

Het adres-veld is een samengestelde waarde, met de velden `address.street`, `address.city` en `address.poscode`. Ook in zo'n samengestelde waarde kun je zoeken:

In [10]:
%%bash
mongo demo --quiet

db.contacts.find({"address.city": "Amsterdam"})

{ "_id" : ObjectId("5d6280bc6f385c47986b6c52"), "name" : "Anna Verschuur", "email" : "anna@hotmail.com", "address" : { "street" : "Noorderkade 102", "city" : "Amsterdam", "postcode" : "1000 AA" } }
{ "_id" : ObjectId("5d6280bc6f385c47986b6c56"), "name" : "Leontien de Bruin ", "email" : [ "lhmdebruin@hotmail.com", "leontien134@tiscalimail.nl" ], "address" : { "street" : "Tulpstraat 17", "city" : "Amsterdam" } }
{ "_id" : ObjectId("5d6280bc6f385c47986b6c58"), "name" : "F.G. Schuitema", "address" : { "street" : "Eikenlaan 23", "city" : "Amsterdam", "postcode" : "1001 AB" } }


**Opdracht** Dit heeft een andere betekenis dan `{"address":{"city": "Amsterdam"}}` . **Ga dit na**. (Waarom zou dat zo zijn?)

In [11]:
%%bash
mongo demo --quiet

db.contacts.find({???}})

2019-08-25T14:36:16.711+0200 E QUERY    [js] SyntaxError: invalid property id @(shell):1:18


**Opdracht.** Je kunt selectie en projectie combineren. 
Maak een query met als resultaat (alleen) de namen van alle contacten met woonplaats Amsterdam.

In [12]:
%%bash
mongo demo --quiet

db.contacts.find({<selectie>}, {<projectie>})

2019-08-25T14:36:17.063+0200 E QUERY    [js] SyntaxError: invalid property id @(shell):1:18


## Selectie en meervoudige waarden

In de voorbeeld-documenten komen documenten voor met meervoudige waarden voor het `email`-veld. Met een enkelvoudig veld in de zoekopdracht vind je ook docementen met zo'n samengestelde waarde:

In [13]:
%%bash
mongo demo --quiet

db.contacts.find({"email": "leontien134@tiscalimail.nl"})

{ "_id" : ObjectId("5d6280bc6f385c47986b6c56"), "name" : "Leontien de Bruin ", "email" : [ "lhmdebruin@hotmail.com", "leontien134@tiscalimail.nl" ], "address" : { "street" : "Tulpstraat 17", "city" : "Amsterdam" } }


## And en or

Als je in het query-document meerdere velden invult, dan voldoen de zoekresultaten aan de "en" (*and*) van deze ingevulde velden.

Een *or* is wat ingewikkelder: je gebruikt daarvoor een `$or`, met een lijst van alternatieven. Voorbeeld:

In [14]:
%%bash
mongo demo --quiet

db.contacts.find({"$or": [{"address.city": "Amsterdam"}, {"address.city": "Rotterdam"} ]})

{ "_id" : ObjectId("5d6280bc6f385c47986b6c52"), "name" : "Anna Verschuur", "email" : "anna@hotmail.com", "address" : { "street" : "Noorderkade 102", "city" : "Amsterdam", "postcode" : "1000 AA" } }
{ "_id" : ObjectId("5d6280bc6f385c47986b6c53"), "name" : "Joop de Zwart", "email" : "zwartejoop@ziggo.nl", "address" : { "street" : "Rozengracht 42", "city" : "Rotterdam" } }
{ "_id" : ObjectId("5d6280bc6f385c47986b6c54"), "name" : "Adrie Vierhuis", "email" : "a34huis@gmail.com", "address" : { "street" : "Beukenweg 12", "city" : "Rotterdam", "postcode" : "2001 BC" }, "tel" : "010-123 123 9" }
{ "_id" : ObjectId("5d6280bc6f385c47986b6c55"), "name" : "Gijs Bennekom", "email" : "gijsbkom@ziggo.nl", "address" : { "street" : "Lijsterlaan 132", "city" : "Rotterdam" } }
{ "_id" : ObjectId("5d6280bc6f385c47986b6c56"), "name" : "Leontien de Bruin ", "email" : [ "lhmdebruin@hotmail.com", "leontien134@tiscalimail.nl" ], "address" : { "street" : "Tulpstraat 17", "city" : "Amsterdam" } }
{ "_id" : Object

## Ontbrekende velden

Zoals we gezien hebben kunnen de documenten in een collection nogal verschillen in de velden die ze definiëren.
Met de query `{field: {$exists: false}}` vind je de documenten zonder `field`.

In [15]:
%%bash
mongo demo --quiet

db.contacts.find({"address.city": {"$exists": false}})

{ "_id" : ObjectId("5d6280bc6f385c47986b6c57"), "name" : "Hans de Boer", "email" : "hdb@example.com", "tel" : "06-1290 8746" }


**Opdracht** vind de documenten zonder postcode.

In [16]:
%%bash
mongo demo --quiet

db.contacts.find({???})

2019-08-25T14:36:18.743+0200 E QUERY    [js] SyntaxError: invalid property id @(shell):1:18


**Opdracht.** vind de documenten **met** postcode. (Wat is het alternatief voor `false`?)

In [17]:
%%bash
mongo demo --quiet

db.contacts.find({???})

2019-08-25T14:36:19.090+0200 E QUERY    [js] SyntaxError: invalid property id @(shell):1:18


## Reguliere expressies

In de query kun je ook een *reguliere expressie* opgeven, om op een ruimere manier strings te matchen.
Zie voor de uitgebreide mogelijkheden de MongoDB-documentatie.

Een voorbeeld hoe je dit in MongoDB gebruikt: zoek alle documenten waarvan de `address.city`-string de substring `dam` bevat. (In dit geval, "Amsterdam" en "Rotterdam").

In [18]:
%%bash
mongo demo --quiet

db.contacts.find({"address.city": {"$regex": /dam/}})

{ "_id" : ObjectId("5d6280bc6f385c47986b6c52"), "name" : "Anna Verschuur", "email" : "anna@hotmail.com", "address" : { "street" : "Noorderkade 102", "city" : "Amsterdam", "postcode" : "1000 AA" } }
{ "_id" : ObjectId("5d6280bc6f385c47986b6c53"), "name" : "Joop de Zwart", "email" : "zwartejoop@ziggo.nl", "address" : { "street" : "Rozengracht 42", "city" : "Rotterdam" } }
{ "_id" : ObjectId("5d6280bc6f385c47986b6c54"), "name" : "Adrie Vierhuis", "email" : "a34huis@gmail.com", "address" : { "street" : "Beukenweg 12", "city" : "Rotterdam", "postcode" : "2001 BC" }, "tel" : "010-123 123 9" }
{ "_id" : ObjectId("5d6280bc6f385c47986b6c55"), "name" : "Gijs Bennekom", "email" : "gijsbkom@ziggo.nl", "address" : { "street" : "Lijsterlaan 132", "city" : "Rotterdam" } }
{ "_id" : ObjectId("5d6280bc6f385c47986b6c56"), "name" : "Leontien de Bruin ", "email" : [ "lhmdebruin@hotmail.com", "leontien134@tiscalimail.nl" ], "address" : { "street" : "Tulpstraat 17", "city" : "Amsterdam" } }
{ "_id" : Object

**Opdracht.** Zoek alle documenten waarvan de naam "Anna" bevat.

In [19]:
%%bash
mongo demo --quiet

db.contacts.find({???})

2019-08-25T14:36:19.838+0200 E QUERY    [js] SyntaxError: invalid property id @(shell):1:18


## Verder

* [Inhoudsopgave](Inhoud.ipynb)